Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import requests
import gzip
from io import BytesIO, StringIO
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras import layers, models
!pip install python-chess
import chess
import chess.pgn
!pip install gym
!pip install gym-chess
import gym
import os
import gym.spaces
import gym_chess
from gym_chess.alphazero.move_encoding import utils, queenmoves, knightmoves, underpromotions
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/CCRL-4040.[1712229].pgn' '/content/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Data* Extracting

In [ ]:
def extract_fen_and_moves(pgn_file_path, num_games_to_read):
    with open(pgn_file_path) as pgn_file:
        fen_moves_list = []
        games_read = 0

        while games_read < num_games_to_read:
            # Read the next line from the PGN file
            line = pgn_file.readline()

            # Break the loop if the end of the file is reached
            if not line:
                break

            # Check if the line starts a new game
            if line.startswith('[Event'):
                # Create a new game
                game = chess.pgn.read_game(pgn_file)

                board = chess.Board()
                fen_moves = []

                # Process moves in the current game
                for move in game.mainline_moves():
                    board.push(move)
                    fen_moves.append((board.fen(), move.uci()))

                fen_moves_list.append(fen_moves)
                games_read += 1

    return fen_moves_list

pgn_file_path = "/content/CCRL-4040.[1712229].pgn"

num_games_to_read = 100

fen_moves_list = extract_fen_and_moves(pgn_file_path, num_games_to_read)

env = gym.make('ChessAlphaZero-v0')
env.reset()

array([[[0, 0, 0, ..., 1, 1, 0],
        [0, 1, 0, ..., 1, 1, 0],
        [0, 0, 1, ..., 1, 1, 0],
        ...,
        [0, 0, 1, ..., 1, 1, 0],
        [0, 1, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0]],

       [[1, 0, 0, ..., 1, 1, 0],
        [1, 0, 0, ..., 1, 1, 0],
        [1, 0, 0, ..., 1, 1, 0],
        ...,
        [1, 0, 0, ..., 1, 1, 0],
        [1, 0, 0, ..., 1, 1, 0],
        [1, 0, 0, ..., 1, 1, 0]],

       [[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0]],

       ...,

       [[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0]],

       [[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 0],
        ...,
        [0, 0, 0, ..., 

Data Formating + Separation

In [ ]:

def get_piece_square_table(piece_type):
    piece_square_tables = {
        chess.PAWN: np.array([
            [0, 0, 0, 0, 0, 0, 0, 0],
            [50, 50, 50, 50, 50, 50, 50, 50],
            [10, 10, 20, 30, 30, 20, 10, 10],
            [5, 5, 10, 25, 25, 10, 5, 5],
            [0, 0, 0, 20, 20, 0, 0, 0],
            [5, -5, -10, 0, 0, -10, -5, 5],
            [5, 10, 10, -20, -20, 10, 10, 5],
            [0, 0, 0, 0, 0, 0, 0, 0]
        ]),
        chess.KNIGHT: np.array([
            [-50, -40, -30, -30, -30, -30, -40, -50],
            [-40, -20, 0, 0, 0, 0, -20, -40],
            [-30, 0, 10, 15, 15, 10, 0, -30],
            [-30, 5, 15, 20, 20, 15, 5, -30],
            [-30, 0, 15, 20, 20, 15, 0, -30],
            [-30, 5, 10, 15, 15, 10, 5, -30],
            [-40, -20, 0, 5, 5, 0, -20, -40],
            [-50, -40, -30, -30, -30, -30, -40, -50]
        ]),
        chess.BISHOP: np.array([
            [-20, -10, -10, -10, -10, -10, -10, -20],
            [-10, 0, 0, 0, 0, 0, 0, -10],
            [-10, 0, 5, 10, 10, 5, 0, -10],
            [-10, 5, 5, 10, 10, 5, 5, -10],
            [-10, 0, 10, 10, 10, 10, 0, -10],
            [-10, 10, 10, 10, 10, 10, 10, -10],
            [-10, 5, 0, 0, 0, 0, 5, -10],
            [-20, -10, -10, -10, -10, -10, -10, -20]
        ]),
        chess.ROOK: np.array([
            [0, 0, 0, 0, 0, 0, 0, 0],
            [5, 10, 10, 10, 10, 10, 10, 5],
            [-5, 0, 0, 0, 0, 0, 0, -5],
            [-5, 0, 0, 0, 0, 0, 0, -5],
            [-5, 0, 0, 0, 0, 0, 0, -5],
            [-5, 0, 0, 0, 0, 0, 0, -5],
            [-5, 0, 0, 0, 0, 0, 0, -5],
            [0, 0, 0, 5, 5, 0, 0, 0]
        ]),
        chess.QUEEN: np.array([
            [-20, -10, -10, -5, -5, -10, -10, -20],
            [-10, 0, 0, 0, 0, 0, 0, -10],
            [-10, 0, 5, 5, 5, 5, 0, -10],
            [-5, 0, 5, 5, 5, 5, 0, -5],
            [0, 0, 5, 5, 5, 5, 0, -5],
            [-10, 5, 5, 5, 5, 5, 0, -10],
            [-10, 0, 5, 0, 0, 0, 0, -10],
            [-20, -10, -10, -5, -5, -10, -10, -20]
        ]),
        chess.KING: np.array([
            [-30, -40, -40, -50, -50, -40, -40, -30],
            [-30, -40, -40, -50, -50, -40, -40, -30],
            [-30, -40, -40, -50, -50, -40, -40, -30],
            [-30, -40, -40, -50, -50, -40, -40, -30],
            [-20, -30, -30, -40, -40, -30, -30, -20],
            [-10, -20, -20, -20, -20, -20, -20, -10],
            [20, 20, 0, 0, 0, 0, 20, 20],
            [20, 30, 10, 0, 0, 10, 30, 20]
        ]),
    }

    return piece_square_tables.get(piece_type, np.zeros((8, 8), dtype=np.int32))

def list_of_legal_moves_for_a_piece(board, position):
  legal_moves = board.legal_moves
  legal_moves_encoded = []
  for move in legal_moves:
    move_uci = move.uci()
    if move_uci[0] == position[0] and move_uci[1] == position[1]:
      if board.turn is chess.BLACK:
        move = utils.rotate(move)
      legal_moves_encoded.append(env.encode(move))
  return legal_moves_encoded


def fen_to_array(fen):
    board = chess.Board(fen)
    board_array = np.zeros((8, 8, 58), dtype=np.uint8)

    # Piece channels
    for square, piece in enumerate(board.piece_map()):
        if board.piece_at(square) is not None:
            i, j = divmod(square, 8)
            piece_index = 'rnbqkp'.index(board.piece_at(square).symbol().lower())
            board_array[i, j, piece_index] = 1

    # Additional channels
    board_array[:, :, 6] = board.turn
    board_array[:, :, 7] = board.has_kingside_castling_rights(chess.WHITE)
    board_array[:, :, 8] = board.has_kingside_castling_rights(chess.BLACK)
    board_array[:, :, 9] = board.has_queenside_castling_rights(chess.WHITE)
    board_array[:, :, 10] = board.has_queenside_castling_rights(chess.BLACK)

    # Material Count
    white_piece_counts = len(board.pieces(chess.PAWN, chess.WHITE)) + 5 * len(board.pieces(chess.ROOK, chess.WHITE)) + 3.1 * len(board.pieces(chess.BISHOP, chess.WHITE))
    + 3 * len(board.pieces(chess.KNIGHT, chess.WHITE)) + 9 * len(board.pieces(chess.QUEEN, chess.WHITE))
    board_array[:, :, 11] = white_piece_counts
    black_piece_counts = len(board.pieces(chess.PAWN, chess.BLACK)) + 5 * len(board.pieces(chess.ROOK, chess.BLACK)) + 3.1 * len(board.pieces(chess.BISHOP, chess.BLACK))
    + 3 * len(board.pieces(chess.KNIGHT, chess.BLACK)) + 9 * len(board.pieces(chess.QUEEN, chess.BLACK))
    board_array[:, :, 12] = black_piece_counts

    # available moves for a chess piece
    for square, piece in enumerate(board.piece_map()):
        if board.piece_at(square) is not None:
            i, j = divmod(square, 8)
            legal_list = list_of_legal_moves_for_a_piece(board,chess.square_name(chess.square(i,j)))
            for n in range(len(legal_list)):
              board_array[i, j, 13 + n] = legal_list[n]

    # Piece Square Tables
    for piece_type in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.QUEEN, chess.KING]:  # Add more piece types as needed
        pst = get_piece_square_table(piece_type)
        for square, value in zip(board.pieces(piece_type, chess.WHITE), pst.flatten()):
            rank, file = chess.square_rank(square), chess.square_file(square)
            board_array[rank, file, 57] = value

    return board_array

def create_dataset(fen_moves_list):
    Y = [[0]*4672]*0
    X = []
    for game_moves in fen_moves_list:
        i = 0
        for fen, move in game_moves:
            board_array = fen_to_array(fen)
            board = chess.Board(fen)
            obj_move = chess.Move.from_uci(move)
            if board.turn is chess.WHITE:
              obj_move = utils.rotate(obj_move)
            y = [0]*4672
            y[env.encode(obj_move)] = 1
            Y.append(y)
            X.append(board_array)
            i = i + 1

    X = np.array(X)
    Y = np.array(Y)

    return X, Y

X, y = create_dataset(fen_moves_list)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Neural Network Creation

In [ ]:
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 58)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(640, activation='relu'))
    model.add(layers.Dense(1280, activation='relu'))
    model.add(layers.Dense(2400, activation='relu'))
    model.add(layers.Dense(3500, activation='relu'))
    model.add(layers.Dense(4672, activation='relu'))
    model.add(layers.Dense(4672, activation='linear'))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

model = create_cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 6, 6, 64)          33472     
                                                                 
 max_pooling2d (MaxPooling2  (None, 3, 3, 64)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 1, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                        

Training

In [ ]:
model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
349/349 [==============================] - 198s 568ms/step - loss: 2.1350e-04 - val_loss: 2.1310e-04
Epoch 2/3
349/349 [==============================] - 243s 696ms/step - loss: 2.1291e-04 - val_loss: 2.1298e-04
Epoch 3/3
349/349 [==============================] - 249s 715ms/step - loss: 2.1255e-04 - val_loss: 2.1221e-04


In [ ]:
def generate_legal_move(model, fen):
  b = []
  board = chess.Board(fen)
  b.append(fen_to_array(fen))
  b = np.array(b)
  a = model.predict(b)
  legal_moves_predicted = []
  all_legal_moves = [env.encode(move) for move in board.legal_moves]
  for move_predicted in range(len(a[0])):
    if a[0][move_predicted] in all_legal_moves:
      legal_moves_predicted.append(a[0][move_predicted])
  best_move_legal_value = 0
  best_move_legal_index = 0
  for i in range(len(legal_moves_predicted)):
    if(best_move_legal_value > legal_moves_predicted[i]):
      best_move_legal_value = legal_moves_predicted[i]
      best_move_legal_index = i

  return all_legal_moves[best_move_legal_index]

def accuracy_check(model,X, y):
  score = 0
  number = 0
  predicted_moves = model.predict(X)
  for move in range(len(predicted_moves)):
    predicted_move = predicted_moves[move][0]
    expected_move = y[move]
    if round(predicted_move) is expected_move:
      score+=1
    number+=1
  return score/number


In [ ]:
rng = np.random.default_rng()
def play_game(model1, model2):
    b = chess.Board()
    b.reset()
    while True:
        b.push_uci(env.decode(generate_legal_move(model1, b.fen())).uci())
        if b.is_game_over():
            if b.outcome().result() == "1-0":
                return 1
            if b.outcome().result() == "0-1":
                return -1
            if b.outcome().result() == "1/2-1/2":
                return 0
        b.push_uci(env.decode(generate_legal_move(model2, b.fen())).uci())
        if b.is_game_over():
            if b.outcome().result() == "1-0":
                return 1
            if b.outcome().result() == "0-1":
                return -1
            if b.outcome().result() == "1/2-1/2":
                return 0
def randomly_evolve(model):
    model_copy = model
    for i in range(13):
        weights = model_copy.layers[i].get_weights()
        for weight in range(len(weights)):
            if rng.integers(0,1) == 1:
                weights[weight] = weights[weight] * (1 - (rng.integers(0, 101)-50)/100)
        model_copy.layers[i].set_weights(weights)
    return model_copy
def procreate_from_pair(model1,model2):
    model3 = model1
    for i in range(13):
        weights1 = model.layers[i].get_weights()
        weights2 = model.layers[i].get_weights()
        for weight in range(len(weights1)):
            if rng.integers(0,1) == 1:
                weights1[weight] = weights1[weight]
            if rng.integers(0,1) == 0:
                weights1[weight] = weights2[weight]
        model3.layers[i].set_weights(weights1)
    return model

def topscore_ind(arr):
  sorted_arr = np.array(arr)
  indexes = np.argsort(sorted_arr)
  indexes = indexes[-5:][::-1]
  return indexes







In [ ]:
models = []
models_selected = []
for i in range(19):
    models.append(randomly_evolve(model))
model_copy = model
models.append(model_copy)
models_score = [0]*20

topscore_ind([100,1,2,3,6,8,10])

array([0, 6, 5, 4, 3])

In [ ]:
for i in range(7):
    for j in range(20):
        for n in range(3):
            models_score[j] = models_score[j] + play_game(models[j], models[rng.integers(0,19)])
    ind = topscore_ind(models_score)
    for j in range(5):
      models_selected.append(models[ind[j]])
    for j in range(5):
      models[j] = models_selected[j]
    for j in range(5):
      models[5 + j] = randomly_evolve(models_selected[j])
    for j in range(4):
      models[10 + j] = procreate_from_pair(models_selected[0],models_selected[rng.integers(1,4)])
    for j in range(4):
      models[14 + j] = procreate_from_pair(models_selected[1],models_selected[rng.integers(1,4)])
    models[18] = procreate_from_pair(models_selected[3],models_selected[rng.integers(1,4)])
    models[19] = procreate_from_pair(models_selected[4],models_selected[rng.integers(1,4)])




In [ ]:
trained_model = models[0]


1/1 [==============================] - 0s 42ms/step


AttributeError: ignored